# Clever Real Estate Analysis v5 - SQL-Based Market Scoring

In [3]:
import pandas as pd
import sqlite3
import re

In [4]:
# Load and clean data
csv_path = r"C:\Users\kaedi\Documents\Python\Portfolio\projects\clever_real_estate\data\real_estate_original.csv"
try:
    df = pd.read_csv(csv_path, encoding='utf-8')
except:
    df = pd.read_csv(csv_path, encoding='latin-1')

df = df.replace('null', pd.NA).replace('NULL', pd.NA).replace('Null', pd.NA)

bool_cols = ['claimed_profile', 'is_active', 'is_cash_home_buyer', 'bbb_accreditation']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].map({'TRUE': True, 'FALSE': False, True: True, False: False}).astype('boolean')

num_cols = ['num_reviews', 'avg_review_rating', 'reviews_last_6_months', 'avg_rating_last_6_months',
            'reviews_last_18_months', 'avg_rating_last_18_months', 'num_months_with_reviews',
            'pct_months_with_review', 'response_rate', 'total_responses', 'months_since_last_review',
            'total_months_active', 'year_first_active', 'num_1_star_reviews', 'bbb_complaints', 
            'google_num_reviews', 'google_avg_review_rating']
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"Loaded {len(df)} companies")

Loaded 4358 companies


In [5]:
db_path = r"C:\Users\kaedi\Documents\Python\Portfolio\projects\clever_real_estate\data\real_estate.db"
conn = sqlite3.connect(db_path)
df.to_sql('companies_raw', conn, if_exists='replace', index=False)

4358

Taking a look to be sure our data is coming through correctly

In [6]:
df.head(15)

,company_id,name,website,phone_number,address,office_type,claimed_profile,is_active,parent_company_id,is_cash_home_buyer,...,bbb_reviews_link,bbb_review_score,bbb_reviews,bbb_complaints,bbb_accreditation,bbb_file_opened,bbb_rating,google_profile_url,google_num_reviews,google_avg_review_rating
0,19d84513-a249-41dc-b305-558965c4d2e4,We Buy Houses Dallas .US,http://www.webuyhousesdallas.us/,NaN,NaN,Local Company,False,True,19d84513-a249-41dc-b305-558965c4d2e4,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
1,0e05d3d7-816d-4274-ad37-16d2db36894b,"Jonas Elber, Real Estate Broker",http://www.jonaselber.com/,(509) 720-8330,"12810 E Nora Ave, Spokane, WA 99206",NaN,False,True,0e05d3d7-816d-4274-ad37-16d2db36894b,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,https://www.google.com/maps/place/Jonas+Elber%...,67.0,4.820896
2,65a2c249-0a2e-4a77-9233-0db96aeef284,The Smith Financial Relief Group,https://allcashbuyers.org/,(440) 837-3753,"227 E SUNSHINE SUITE 228, SPRINGFIELD, MO 65807",NaN,False,False,65a2c249-0a2e-4a77-9233-0db96aeef284,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,https://www.google.com/maps/place/Smith+Financ...,3.0,5.000000
3,92e82a5e-160d-4852-adf8-a30e2ad993bc,Baker LA Mortgage Note Buyers,https://www.cashnoteusa.com/real-estate-mortga...,(225) 372-5468,NaN,NaN,False,True,90d16122-e30c-4cf5-9415-c9e08222e33a,True,...,https://www.bbb.org/us/la/baker/profile/home-b...,NaN,NaN,NaN,False,2020-01-03,A,NaN,NaN,NaN
4,f60ca260-203d-4e61-8605-012bdc8965a4,Mike Buys Houses Fast,https://www.mike-buys-houses-fast.com/,(757) 255-5505,NaN,Local Company,False,True,f60ca260-203d-4e61-8605-012bdc8965a4,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
5,24beb7cb-15ed-4753-837c-ce83341b33b5,Favor Home Solutions,https://favorhomesolutions.com/north-carolina/...,(336) 438-8608,"717 Green Valley Rd Suite 200 #575, Greensboro...",Local Company,False,True,24beb7cb-15ed-4753-837c-ce83341b33b5,True,...,https://www.bbb.org/us/tn/murfreesboro/profile...,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
6,2484124d-499a-4606-b04d-faa16321af0d,We Buy Houses Tucson AZ,http://www.webuyhousestucsonaz.com/,(520) 302-4988,NaN,NaN,False,False,2484124d-499a-4606-b04d-faa16321af0d,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
7,600a1575-117f-4f2e-8bc1-7101134a8f47,California Cash Buyers,https://www.californiacashbuyer.com/,(415) 384-9992,NaN,Local Company,False,True,600a1575-117f-4f2e-8bc1-7101134a8f47,True,...,https://www.bbb.org/us/ca/san-bruno/profile/re...,NaN,NaN,NaN,True,2019-06-19,A+,NaN,NaN,NaN
8,670cd891-98ea-461e-869c-4caa4c4f6f37,Your Trusted Home Buyer Orlando,https://www.yourtrustedhomebuyer.com/we-buy-ho...,(855) 483-2097,NaN,Local Company,False,True,670cd891-98ea-461e-869c-4caa4c4f6f37,True,...,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
9,e22fd275-87f8-47b8-9b1f-18133d31281a,SLO Cash Buyer,https://www.slocashbuyer.com/,(805) 779-3472,NaN,Local Company,False,True,e22fd275-87f8-47b8-9b1f-18133d31281a,True,...,https://www.bbb.org/us/ca/san-luis-obispo/prof...,NaN,NaN,NaN,True,2022-04-27,A+,https://www.google.com/maps/place/SLO+Cash+Buy...,8.0,5.000000


We want to have this dataset be easily scored per market for the final use case so we'll fill in gaps wherever we can and ultimately explode the data to have 1 row per market, allowing a company to be represented in as many markets as appropriate

In [8]:
def split_on_commas(val):
    """Split a comma-separated string into clean items. Does NOT split on spaces."""
    if pd.isna(val):
        return []
    s = str(val).strip()
    if not s or s.lower() in {"null", "none", "nan"}:
        return []
    return [p.strip() for p in s.split(",") if p.strip()]

def extract_city_state_from_address(addr):
    """
    Extract City + 2-letter State from common US addresses like:
    '717 Green Valley Rd..., Greensboro, NC 27408'
    Returns 'City, ST' or None.
    """
    if pd.isna(addr):
        return None
    s = str(addr).strip()
    if not s:
        return None

    m = re.search(r",\s*([^,]+?)\s*,\s*([A-Z]{2})\b(?:\s*\d{5}(?:-\d{4})?)?\s*$", s)
    if m:
        city = m.group(1).strip()
        st = m.group(2).strip()
        return f"{city}, {st}"
    return None

# Explode markets - include ALL columns from original dataset
rows = []
for _, row in df.iterrows():
    markets = []

    # 1) Cities -> City, State (using states field if present)
    cities = split_on_commas(row.get("cities"))
    if cities:
        state = row.get("states", "")
        if pd.notna(state) and str(state).strip():
            markets = [f"{c}, {str(state).strip()}" for c in cities]
        else:
            markets = cities[:]  # no state available, keep city only

    # 2) Counties -> "X County, State"
    if not markets:
        counties = split_on_commas(row.get("counties"))
        if counties:
            state = row.get("states", "")
            if pd.notna(state) and str(state).strip():
                markets = [f"{c} County, {str(state).strip()}" for c in counties]
            else:
                markets = [f"{c} County" for c in counties]

    # 3) States list (can be "North Carolina, South Carolina")
    if not markets:
        markets = split_on_commas(row.get("states"))

    # 4) location_tag fallback
    if not markets and pd.notna(row.get("location_tag")) and str(row.get("location_tag")).strip():
        markets = [str(row.get("location_tag")).strip()]

    # 5) NEW: address-derived fallback (City, ST) if still nothing
    if not markets:
        addr_market = extract_city_state_from_address(row.get("address"))
        if addr_market:
            markets = [addr_market]

    # 6) Unknown
    if not markets:
        markets = ["Unknown"]

    for m in markets:
        row_data = {"company_id": row["company_id"], "company_name": row["name"], "market": m}
        for col in df.columns:
            if col not in ["company_id", "name"]:
                row_data[col] = row.get(col)
        rows.append(row_data)

df_markets = pd.DataFrame(rows)
df_markets.to_sql("market_company", conn, if_exists="replace", index=False)
print(f"Created market_company: {len(df_markets)} rows, {df_markets['market'].nunique()} markets")

# Optional QA: spot-check a known multi-city row explodes into multiple rows
# print(df_markets[df_markets["company_name"].str.contains("California Cash Buyers", na=False)][["company_name","market"]].head(50))

Created market_company: 26891 rows, 9443 markets


Looking good! We have gone from 4k rows to 27k rows

In [9]:
query = """
SELECT * FROM market_company LIMIT 15
"""
dfview = pd.read_sql_query(query, conn)
dfview

,company_id,company_name,market,website,phone_number,address,office_type,claimed_profile,is_active,parent_company_id,...,bbb_reviews_link,bbb_review_score,bbb_reviews,bbb_complaints,bbb_accreditation,bbb_file_opened,bbb_rating,google_profile_url,google_num_reviews,google_avg_review_rating
0,19d84513-a249-41dc-b305-558965c4d2e4,We Buy Houses Dallas .US,"Dallas, Texas",http://www.webuyhousesdallas.us/,None,None,Local Company,0,1,19d84513-a249-41dc-b305-558965c4d2e4,...,None,None,None,None,NaN,None,None,None,NaN,NaN
1,19d84513-a249-41dc-b305-558965c4d2e4,We Buy Houses Dallas .US,"Frisco, Texas",http://www.webuyhousesdallas.us/,None,None,Local Company,0,1,19d84513-a249-41dc-b305-558965c4d2e4,...,None,None,None,None,NaN,None,None,None,NaN,NaN
2,0e05d3d7-816d-4274-ad37-16d2db36894b,"Jonas Elber, Real Estate Broker","Spokane, Washington",http://www.jonaselber.com/,(509) 720-8330,"12810 E Nora Ave, Spokane, WA 99206",None,0,1,0e05d3d7-816d-4274-ad37-16d2db36894b,...,None,None,None,None,NaN,None,None,https://www.google.com/maps/place/Jonas+Elber%...,67.0,4.820896
3,65a2c249-0a2e-4a77-9233-0db96aeef284,The Smith Financial Relief Group,"Springfield, Missouri",https://allcashbuyers.org/,(440) 837-3753,"227 E SUNSHINE SUITE 228, SPRINGFIELD, MO 65807",None,0,0,65a2c249-0a2e-4a77-9233-0db96aeef284,...,None,None,None,None,NaN,None,None,https://www.google.com/maps/place/Smith+Financ...,3.0,5.000000
4,92e82a5e-160d-4852-adf8-a30e2ad993bc,Baker LA Mortgage Note Buyers,Unknown,https://www.cashnoteusa.com/real-estate-mortga...,(225) 372-5468,None,None,0,1,90d16122-e30c-4cf5-9415-c9e08222e33a,...,https://www.bbb.org/us/la/baker/profile/home-b...,None,None,None,0.0,2020-01-03,A,None,NaN,NaN
5,f60ca260-203d-4e61-8605-012bdc8965a4,Mike Buys Houses Fast,"Chesapeake, Virginia",https://www.mike-buys-houses-fast.com/,(757) 255-5505,None,Local Company,0,1,f60ca260-203d-4e61-8605-012bdc8965a4,...,None,None,None,None,NaN,None,None,None,NaN,NaN
6,24beb7cb-15ed-4753-837c-ce83341b33b5,Favor Home Solutions,"Greensboro, NC",https://favorhomesolutions.com/north-carolina/...,(336) 438-8608,"717 Green Valley Rd Suite 200 #575, Greensboro...",Local Company,0,1,24beb7cb-15ed-4753-837c-ce83341b33b5,...,https://www.bbb.org/us/tn/murfreesboro/profile...,None,None,None,NaN,None,None,None,NaN,NaN
7,2484124d-499a-4606-b04d-faa16321af0d,We Buy Houses Tucson AZ,Unknown,http://www.webuyhousestucsonaz.com/,(520) 302-4988,None,None,0,0,2484124d-499a-4606-b04d-faa16321af0d,...,None,None,None,None,NaN,None,None,None,NaN,NaN
8,600a1575-117f-4f2e-8bc1-7101134a8f47,California Cash Buyers,"Alameda, California",https://www.californiacashbuyer.com/,(415) 384-9992,None,Local Company,0,1,600a1575-117f-4f2e-8bc1-7101134a8f47,...,https://www.bbb.org/us/ca/san-bruno/profile/re...,None,None,None,1.0,2019-06-19,A+,None,NaN,NaN
9,600a1575-117f-4f2e-8bc1-7101134a8f47,California Cash Buyers,"Antioch, California",https://www.californiacashbuyer.com/,(415) 384-9992,None,Local Company,0,1,600a1575-117f-4f2e-8bc1-7101134a8f47,...,https://www.bbb.org/us/ca/san-bruno/profile/re...,None,None,None,1.0,2019-06-19,A+,None,NaN,NaN


We ran the above cleaning and housekeeping steps in python but I prefer SQL for analysis so we'll use that going forward to generate scores. Had to rework some of it because I'm used to the slick features of snowflake SQL, but it should read ok in the old SQLITE syntax

In [10]:
# SQL scoring
sql = """
DROP TABLE IF EXISTS market_company_scored;
CREATE TABLE market_company_scored AS
WITH base_reviews AS (
    SELECT
        company_id,
        company_name,
        market,
        is_active,
        is_cash_home_buyer,

        /* Best available rating */
        COALESCE(
            NULLIF(google_avg_review_rating, 0),
            NULLIF(avg_review_rating, 0),
            NULLIF(avg_rating_last_18_months, 0),
            NULLIF(avg_rating_last_6_months, 0)
        ) AS best_rating,

        /* Best available review count */
        COALESCE(
            NULLIF(google_num_reviews, 0),
            NULLIF(num_reviews, 0),
            NULLIF(reviews_last_18_months, 0),
            NULLIF(reviews_last_6_months, 0)
        ) AS best_reviews,

        months_since_last_review,
        reviews_last_6_months,
        pct_months_with_review,
        response_rate,
        total_responses,
        year_first_active,
        total_months_active,
        bbb_rating,
        bbb_accreditation,
        bbb_complaints,
        website,
        phone_number,
        address,
        google_profile_url

    FROM market_company
),
-- Necessary because legacy SQL won't allow us to use the 
bay_prep AS (
    SELECT
        *,
        CAST(best_reviews AS REAL) AS v,
        4.3 AS prior_mean, -- derived from all scores in the dataset, setting everyone on equal default footing
        20.0 AS prior_weight -- helps out smaller companies with less reviews, provides moderate smoothing
    FROM base_reviews
),
bayesian AS (
    SELECT
        *,
        v,
        prior_mean,
        prior_weight,

        CASE
            WHEN best_reviews > 0 THEN
                (v / (v + prior_weight)) * best_rating +
                (prior_weight / (v + prior_weight)) * prior_mean
            ELSE NULL
        END AS bayes_rating

    FROM bay_prep
)

SELECT
    company_id,
    company_name,
    market,
    is_active,
    is_cash_home_buyer,
    best_rating,
    best_reviews,
    bayes_rating,

    /* ---------------------------
       Customer Experience Score
    ---------------------------- */
    CASE
        WHEN best_reviews > 0 THEN
            0.75 *
                MIN(
                    MAX((bayes_rating - 3.0) / 2.0, 0), -- 3 is not a great score, treat it as the floor, better rating closer to 5
                    1
                )
            +
            0.25 *
                (LOG(1.0 + best_reviews) / LOG(1.0 + 604.0)) -- max number of reviews
        ELSE NULL
    END AS customer_experience,

    /* ---------------------------
       Activity Signals
    ---------------------------- */

    CASE
        WHEN months_since_last_review IS NOT NULL THEN
            1.0 - MIN(months_since_last_review / 24.0, 1.0) -- we're considering more than 24 months stale
        ELSE NULL
    END AS recency,

    CASE
        WHEN reviews_last_6_months > 0 THEN
            LOG(1.0 + reviews_last_6_months) / LOG(1.0 + 118.0) -- max number of recent reviews
        ELSE NULL
    END AS velocity,

    pct_months_with_review AS consistency,

    CASE
        WHEN response_rate IS NOT NULL THEN
            response_rate *
            MIN(1.0, COALESCE(total_responses, 0) / 20.0) -- high response rate is considered better the more responses you have, fully eligible for best score at 20 
        ELSE NULL
    END AS responsiveness,

    /* ---------------------------
       Age / Credibility
    ---------------------------- */

    CASE
        WHEN year_first_active IS NOT NULL THEN
            MIN((2026 - year_first_active) / 10.0, 1.0)
        WHEN total_months_active IS NOT NULL THEN
            MIN((total_months_active / 12.0) / 10.0, 1.0)
        ELSE NULL
    END AS age_score,

    CASE bbb_rating
        WHEN 'A+' THEN 1.00
        WHEN 'A'  THEN 0.95
        WHEN 'A-' THEN 0.90
        WHEN 'B+' THEN 0.85
        WHEN 'B'  THEN 0.80
        WHEN 'B-' THEN 0.75
        WHEN 'C+' THEN 0.70
        WHEN 'C'  THEN 0.65
        WHEN 'C-' THEN 0.60
        WHEN 'D+' THEN 0.55
        WHEN 'D'  THEN 0.50
        WHEN 'D-' THEN 0.45
        WHEN 'F'  THEN 0.10
        ELSE NULL
    END AS bbb_letter_score,

    /* ---------------------------
       Presence Score
    ---------------------------- */

    (
        CASE WHEN website IS NOT NULL AND website <> '' THEN 1 ELSE 0 END +
        CASE WHEN phone_number IS NOT NULL AND phone_number <> '' THEN 1 ELSE 0 END +
        CASE WHEN address IS NOT NULL AND address <> '' THEN 1 ELSE 0 END +
        CASE WHEN google_profile_url IS NOT NULL AND google_profile_url <> '' THEN 1 ELSE 0 END
    ) / 4.0 AS presence,

    /* ---------------------------
       Raw Fields for Debug / QA
    ---------------------------- */

    months_since_last_review,
    reviews_last_6_months,
    pct_months_with_review,
    response_rate,
    total_responses,
    year_first_active,
    total_months_active,
    bbb_rating,
    bbb_accreditation,
    bbb_complaints,
    website,
    phone_number,
    address,
    google_profile_url

FROM bayesian;
"""
conn.executescript(sql)
print("Created market_company_scored")

Created market_company_scored


The below only needed to be run once to add columns to hold our scores, we comment it out for the rest of development

In [ ]:
# conn.executescript("""
# ALTER TABLE market_company_scored ADD COLUMN local_activity REAL;
# ALTER TABLE market_company_scored ADD COLUMN credibility REAL;
# ALTER TABLE market_company_scored ADD COLUMN confidence_multiplier REAL;
# ALTER TABLE market_company_scored ADD COLUMN final_score REAL;
# ALTER TABLE market_company_scored ADD COLUMN is_excluded INTEGER;
# ALTER TABLE market_company_scored ADD COLUMN penalty REAL;
# ALTER TABLE market_company_scored ADD COLUMN final_score_adjusted REAL;
# """)

In [14]:
conn.executescript("""
DROP VIEW IF EXISTS v_market_company_scored_full;

CREATE VIEW v_market_company_scored_full AS
WITH joined AS (
  SELECT
    mc.*,
    sc.best_rating, sc.best_reviews, sc.customer_experience,
    sc.recency, sc.velocity, sc.consistency, sc.responsiveness,
    sc.age_score, sc.bbb_letter_score, sc.presence
  FROM market_company mc
  LEFT JOIN market_company_scored sc
    ON sc.company_id = mc.company_id
   AND sc.market      = mc.market
),
c AS (
  SELECT
    j.*,

    /* derived components */
    COALESCE(recency,0)*0.35 + COALESCE(velocity,0)*0.25 + COALESCE(consistency,0)*0.25 + COALESCE(responsiveness,0)*0.15 AS local_activity,
    COALESCE(age_score,0)*0.45 + COALESCE(bbb_letter_score,0)*0.35 + COALESCE(presence,0)*0.20 AS credibility,

    0.85 + 0.15 * (
      (CASE WHEN best_rating IS NOT NULL AND best_reviews IS NOT NULL THEN 1 ELSE 0 END
     + CASE WHEN bbb_rating IS NOT NULL OR bbb_accreditation=1 OR bbb_complaints IS NOT NULL THEN 1 ELSE 0 END
     + CASE WHEN months_since_last_review IS NOT NULL OR reviews_last_6_months IS NOT NULL THEN 1 ELSE 0 END
     + CASE WHEN year_first_active IS NOT NULL OR total_months_active IS NOT NULL THEN 1 ELSE 0 END
      ) / 4.0
    ) AS confidence_multiplier,

    CASE
      WHEN COALESCE(is_active,0)=0 OR COALESCE(is_cash_home_buyer,0)=0 OR (best_rating<=2.5 AND COALESCE(num_reviews,0)>=10)
      THEN 1 ELSE 0
    END AS is_excluded

  FROM joined j
),
final_score AS (
  SELECT
    c.*,

    CASE
      WHEN customer_experience IS NOT NULL
      THEN 100.0 * (0.40*credibility + 0.35*customer_experience + 0.25*local_activity) * confidence_multiplier
      ELSE 100.0 * (0.40*credibility + 0.25*local_activity) * confidence_multiplier
    END AS final_score,

    (0
     + CASE WHEN num_1_star_reviews IS NOT NULL AND COALESCE(num_reviews,0)>0
             AND (CAST(num_1_star_reviews AS REAL)/CAST(num_reviews AS REAL))>0.25 AND num_reviews>=10
            THEN 10 ELSE 0 END
     + CASE WHEN months_since_last_review>=24 THEN 5 ELSE 0 END
    ) AS penalty

  FROM c
)
SELECT
  f.*,
  CASE WHEN final_score IS NULL THEN NULL ELSE MAX(0, final_score - COALESCE(penalty,0)) END AS final_score_adjusted
FROM final_Score f;
""")

In [15]:
mfquery = """
SELECT * FROM v_market_company_scored_full LIMIT 15
"""
mfdf = pd.read_sql_query(mfquery, conn)
mfdf

,company_id,company_name,market,website,phone_number,address,office_type,claimed_profile,is_active,parent_company_id,...,age_score,bbb_letter_score,presence,local_activity,credibility,confidence_multiplier,is_excluded,final_score,penalty,final_score_adjusted
0,19d84513-a249-41dc-b305-558965c4d2e4,We Buy Houses Dallas .US,"Dallas, Texas",http://www.webuyhousesdallas.us/,None,None,Local Company,0,1,19d84513-a249-41dc-b305-558965c4d2e4,...,NaN,NaN,0.25,0.000000,0.0500,0.8500,0,1.700000,0,1.700000
1,19d84513-a249-41dc-b305-558965c4d2e4,We Buy Houses Dallas .US,"Frisco, Texas",http://www.webuyhousesdallas.us/,None,None,Local Company,0,1,19d84513-a249-41dc-b305-558965c4d2e4,...,NaN,NaN,0.25,0.000000,0.0500,0.8500,0,1.700000,0,1.700000
2,0e05d3d7-816d-4274-ad37-16d2db36894b,"Jonas Elber, Real Estate Broker","Spokane, Washington",http://www.jonaselber.com/,(509) 720-8330,"12810 E Nora Ave, Spokane, WA 99206",None,0,1,0e05d3d7-816d-4274-ad37-16d2db36894b,...,1.0,NaN,1.00,0.322012,0.6500,0.9625,0,59.811708,0,59.811708
3,65a2c249-0a2e-4a77-9233-0db96aeef284,The Smith Financial Relief Group,"Springfield, Missouri",https://allcashbuyers.org/,(440) 837-3753,"227 E SUNSHINE SUITE 228, SPRINGFIELD, MO 65807",None,0,0,65a2c249-0a2e-4a77-9233-0db96aeef284,...,0.4,NaN,1.00,0.022821,0.3800,0.9625,1,34.577966,5,29.577966
4,92e82a5e-160d-4852-adf8-a30e2ad993bc,Baker LA Mortgage Note Buyers,Unknown,https://www.cashnoteusa.com/real-estate-mortga...,(225) 372-5468,None,None,0,1,90d16122-e30c-4cf5-9415-c9e08222e33a,...,NaN,0.95,0.50,0.000000,0.4325,0.8875,0,15.353750,0,15.353750
5,f60ca260-203d-4e61-8605-012bdc8965a4,Mike Buys Houses Fast,"Chesapeake, Virginia",https://www.mike-buys-houses-fast.com/,(757) 255-5505,None,Local Company,0,1,f60ca260-203d-4e61-8605-012bdc8965a4,...,NaN,NaN,0.50,0.000000,0.1000,0.8500,0,3.400000,0,3.400000
6,24beb7cb-15ed-4753-837c-ce83341b33b5,Favor Home Solutions,"Greensboro, NC",https://favorhomesolutions.com/north-carolina/...,(336) 438-8608,"717 Green Valley Rd Suite 200 #575, Greensboro...",Local Company,0,1,24beb7cb-15ed-4753-837c-ce83341b33b5,...,0.4,NaN,0.75,0.697732,0.3300,0.9625,0,56.255934,0,56.255934
7,24beb7cb-15ed-4753-837c-ce83341b33b5,Favor Home Solutions,"Greensboro, NC",https://favorhomesolutions.com/north-carolina/...,(336) 438-8608,"717 Green Valley Rd Suite 200 #575, Greensboro...",Local Company,0,1,24beb7cb-15ed-4753-837c-ce83341b33b5,...,0.4,1.00,0.75,0.697732,0.6800,0.9625,0,69.730934,0,69.730934
8,2484124d-499a-4606-b04d-faa16321af0d,We Buy Houses Tucson AZ,Unknown,http://www.webuyhousestucsonaz.com/,(520) 302-4988,None,None,0,0,2484124d-499a-4606-b04d-faa16321af0d,...,NaN,NaN,0.50,0.000000,0.1000,0.8500,1,3.400000,0,3.400000
9,600a1575-117f-4f2e-8bc1-7101134a8f47,California Cash Buyers,"Alameda, California",https://www.californiacashbuyer.com/,(415) 384-9992,None,Local Company,0,1,600a1575-117f-4f2e-8bc1-7101134a8f47,...,NaN,1.00,0.50,0.000000,0.4500,0.8875,0,15.975000,0,15.975000


In [21]:
mjquery = """
WITH eligible AS (
  SELECT
    market,
    company_id,
    company_name,
    website,
    phone_number,
    best_rating,
    best_reviews,
    customer_experience,
    credibility,
    local_activity,
    confidence_multiplier,
    penalty,
    final_score_adjusted

  FROM v_market_company_scored_full
  WHERE COALESCE(is_excluded, 0) = 0
    AND final_score_adjusted IS NOT NULL
)
SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY market ORDER BY final_score_adjusted DESC) AS market_rank
FROM eligible
WHERE market = 'Dallas, Texas'
ORDER BY market_rank 
"""
mjdf = pd.read_sql_query(mjquery, conn)
mjdf

,market,company_id,company_name,website,phone_number,best_rating,best_reviews,customer_experience,credibility,local_activity,confidence_multiplier,penalty,final_score_adjusted,market_rank
0,"Dallas, Texas",83149b17-d61d-4614-9d26-f171ff3aef60,Big State Home Buyers,https://www.bigstatehomebuyers.com/,(713) 322-8803,4.738916,203.0,0.844901,1.000,0.635254,1.00,0,85.452900,1
1,"Dallas, Texas",38265613-42c6-4485-99ad-2292c14dfef0,Metroplex Homebuyers,https://metroplexhomebuyers.com/,(972) 487-7653,4.860465,129.0,0.859446,1.000,0.560493,1.00,0,84.092937,2
2,"Dallas, Texas",6bcb0f29-34ef-4c2b-b3b0-c42583edf3ff,Orchard,https://Orchard.com,(844) 515-9880,4.353674,803.0,0.768238,0.815,0.888848,1.00,0,81.709517,3
3,"Dallas, Texas",61235d47-1694-4f91-92cc-14f1d5815318,Home Buying Guys,http://www.homebuyingguys.com/,(866) 847-3565,4.948718,78.0,0.851665,0.910,0.593970,1.00,0,81.057514,4
4,"Dallas, Texas",54e3ad8e-4cdf-44d7-bb7e-440f0ecf48f4,TX Cash Home Buyers,https://www.txcashhomebuyers.com/,(281) 595-7550,4.790698,43.0,0.760794,0.865,0.668765,1.00,0,77.946921,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,"Dallas, Texas",414fe30e-46dc-48c7-9824-6e67ba77f7e9,We Buy Houses in DFW TX,https://www.webuyhousesindfwtx.com/,(817) 381-6290,NaN,NaN,NaN,0.100,0.000000,0.85,0,3.400000,104
104,"Dallas, Texas",976242ee-dbbe-40d3-b5bc-a9688ed332ef,We Buy Dallas Houses,https://www.webuydallashouses.com/,(254) 272-3743,NaN,NaN,NaN,0.100,0.000000,0.85,0,3.400000,105
105,"Dallas, Texas",480be679-9856-4234-a361-6b9b9e138b62,Cash Home Offer USA,https://www.cashhomeofferusa.com/,(888) 724-9603,NaN,NaN,NaN,0.100,0.000000,0.85,0,3.400000,106
106,"Dallas, Texas",7f7d0fc0-410f-439b-a3b2-067fad31609f,Simple House Solutions,https://simplehousedfw.com/,(972) 876-3131,NaN,NaN,NaN,0.100,0.000000,0.85,0,3.400000,107


In [ ]:
# Close connection when done

conn.close()
print("\nDone!")